In [ ]:
library(devtools)
devtools::install_local('/home/jovyan/gganimate-0.1.1.tar.gz')
devtools::install_local('/home/jovyan/ggraph-d15fd149babe9ad32316474b9a178e019f376ba6.zip')
library(gganimate)
library(ggraph)
library(igraph)
library(RColorBrewer)

In [ ]:
# Originally for data from http://konect.uni-koblenz.de/networks/sociopatterns-infectious
# as part of a museum exhibit on spread of infections http://www.sociopatterns.org/deployments/infectious-sociopatterns/
# and the paper http://www.sociopatterns.org/publications/whats-in-a-crowd-analysis-of-face-to-face-behavioral-networks/

# Code adapted from https://gist.github.com/thomasp85/eee48b065ff454e390e1
# and from https://gist.github.com/jalapic/612036977d9f9c773107681bc4a46d58

infect <- read.table('/home/jovyan/networkDynamicsLabels.txt', skip = 0, sep = ' ', stringsAsFactors = FALSE)
infect$V3 <- NULL 
names(infect) <- c('from', 'to', 'time')
infect$timebins <- as.numeric(cut(infect$time, breaks = 150)) # lower means more bursty, i.e. breaks = 10

# We want that nice fading effect so we need to add extra data for the trailing
infectAnim <- lapply(1:10, function(i) {infect$timebins  <- infect$timebins + i; infect$delay <- i; infect})
infect$delay <- 0
infectAnim <- rbind(infect, do.call(rbind, infectAnim))

infectGraph <- graph_from_data_frame(infectAnim, directed = F)

# We use only original data for the layout
subGr <- subgraph.edges(infectGraph, which(E(infectGraph)$delay == 0))
V(subGr)$degree <- degree(subGr)
V(subGr)$group <- cluster_louvain(subGr)$membership
lay <- createLayout(subGr, 'igraph', algorithm = 'fr') # change spatial layout of network

# Then we reassign the full graph with edge trails
attr(lay, 'graph') <- infectGraph

# Now we create the graph with timebins as frame
p <- ggraph(data = lay, layout = 'fr', aes(frame = timebins)) + 
  geom_node_point(size = .025, col = "white") + # change size & color of inactive nodes
  geom_node_point(aes(alpha=0.6), size = .025, colour = factor(lay$group), show.legend = FALSE) + # change size & color of active nodes
  # geom_edge_link0(aes(frame = timebins, alpha = delay, width = delay), edge_colour = '#dccf9f') + 
  geom_edge_link0(aes(frame = timebins, alpha = delay, width = delay, colour = factor(node1.group)), data = gEdges(nodePar = 'group'), show.legend = FALSE) +
  # geom_edge_link0(aes(frame = timebins, alpha = delay, width = delay, colour = node1.degree), data = gEdges(nodePar = 'degree'), show.legend = FALSE) +
  scale_edge_alpha(range = c(1, 0), guide = 'none') + 
  scale_edge_width(range = c(0.25, 0.75), trans = 'exp', guide = 'none') + # change edge width
  scale_size(guide = 'none') + 
  expand_limits(x = c(min(lay$x), max(lay$x)), y = c(min(lay$y), max(lay$y))) +
  ggforce::theme_no_axes() + 
  theme(plot.background = element_rect(fill = '#000000'), # change background color
        panel.background = element_blank(), 
        panel.border = element_blank(), 
        plot.title = element_text(color = '#cecece'))

infect

In [ ]:
# Note if the animation surpasses the memory usage of the notebook, it will crash.
# reduce memory by changing image size, image resolution, making network smaller, 
# changing interval, and/or changing breaks

# And then we animate
animation::ani.options(interval=0.01) # change speed of frame transitions

gganim <- gganimate(p, '/home/jovyan/sparkingCuriosity.gif', title_frame = FALSE,
         ani.width = 400, ani.height = 400, res=100) # change image size and resolution
